# Process (Filter) Data Extracted From Quotebank For Disaster

**Imports**

In [318]:
import pandas as pd
import re # Used for verifying and running small tests on regex patterns
import datetime

from disaster_extr_constants import *
from disaster_extr_helpers import *

**Define parameters for dataset to be filtered**

In [352]:
YEAR = 2017
disaster_type = 'storm'
disaster_YEAR_pos = storm_tags_2017_pos
disaster_YEAR_neg = storm_tags_2017_neg
file_path = 'data/'+str(YEAR)+'_'+disaster_type+'_climate_processed_csv.bz2'

**Load EMDAT for date retrieval and extracted quote set (which still needs to be filtered)**

In [353]:
data = 'data/emdat_processed.csv'
parse_dates = ['StartDate', 'EndDate']
df_emdat = pd.read_csv(data, index_col="Dis No", parse_dates = parse_dates)

**Extracted quotes are from larger time interval than disaster itself to account for evolution of discussion around climate change. Specifically, quotes are extracted in the interval [start - 21 days, end + 21 days]. Note that if the disaster lasted more than 30 days the interval is [start - 21 days, end].**

In [354]:
disaster_df_quotes = pd.read_csv(file_path, parse_dates = ['date'], compression='bz2')

**Apply date retrieval (special case for 2020 because StartDate is in 2019)**

In [400]:
if disaster_type == 'heat_wave':
    
    if YEAR == 2020:
        df_heat_wave = get_df_disaster(df_emdat, HEAT_WAVES_2020, HEAT_WAVES_2020_val)
        df_heat_wave_bounds = retrieve_bounding_dates(df_heat_wave)
    else:
        df_heat_wave = get_df_disaster(df_emdat, HEAT_WAVES, HEAT_WAVES_val)
        df_heat_wave_bounds = retrieve_bounding_dates(df_heat_wave)
        
    disaster_df = df_heat_wave
    disaster_df_bounds = df_heat_wave_bounds
        
elif disaster_type == 'storm':
    
    if YEAR == 2020:
        df_storm = get_df_disaster(df_emdat, STORMS_2020, STORMS_2020_val)
        df_storm_bounds = retrieve_bounding_dates(df_storm)
    else:
        df_storm = get_df_disaster(df_emdat, STORMS, STORMS_val)
        df_storm_bounds = retrieve_bounding_dates(df_storm)

    disaster_df = df_storm
    disaster_df_bounds = df_storm_bounds
    
if YEAR == 2020:
    lookupYEAR = YEAR-1
else:
    lookupYEAR = YEAR
    

start_YEAR, end_YEAR = disaster_df_bounds.loc[lookupYEAR].MinStartDate, disaster_df_bounds.loc[lookupYEAR].MaxEndDate

**Overview over all disasters for a given disaster type (note: separate table for 2020)**

In [401]:
disaster_df[['Type', 'Subtype','Name', 'Country','StartDate','EndDate']]

,Type,Subtype,Name,Country,StartDate,EndDate
Dis No,,,,,,
2015-0470-MEX,Storm,Tropical cyclone,Hurricane Patricia,Mexico,2015-10-22,2015-10-28
2016-0041-FJI,Storm,Tropical cyclone,Cyclone Winston,Fiji,2016-02-20,2016-02-21
2017-0362-USA,Storm,Tropical cyclone,Hurricane Harvey,United States of America (the),2017-08-25,2017-08-29
2018-0341-CHN,Storm,Tropical cyclone,Typhoon Mangkut (Ompong),China,2018-09-10,2018-09-18
2018-0342-USA,Storm,Tropical cyclone,Hurricane Florence,United States of America (the),2018-09-12,2018-09-18
2018-0341-PHL,Storm,Tropical cyclone,Typhoon Mangkut (Ompong),Philippines (the),2018-09-16,2018-09-16
2018-0341-HKG,Storm,Tropical cyclone,Typhoon Mangkut (Ompong),Hong Kong,2018-09-17,2018-09-17
2019-0492-JPN,Storm,Tropical cyclone,Tropical cylone 'Hagibis',Japan,2019-10-12,2019-10-17


**Overview over start and end dates (when a particluar disaster contains several entries pick the earliest StartDate and latest EndDate)**

In [402]:
disaster_df_bounds

,MinStartDate,MaxEndDate
Year,,
2015,2015-10-22,2015-10-28
2016,2016-02-20,2016-02-21
2017,2017-08-25,2017-08-29
2018,2018-09-10,2018-09-18
2019,2019-10-12,2019-10-17


In [403]:
print("Disaster [{}] Dates: {} --- {}".format(
        disaster_type,
        start_YEAR.strftime("%Y-%m-%d"), 
        end_YEAR.strftime("%Y-%m-%d")))

Disaster [storm] Dates: 2017-08-25 --- 2017-08-29


**If duration of disaster is less than a month, add 10 days to end date**

In [404]:
# Add 10 days to end date
one_month = datetime.timedelta(days=31)
if end_YEAR - start_YEAR < one_month:
    ten_days = datetime.timedelta(days=10)
    end_YEAR += ten_days

In [405]:
print("Used extraction dates: {} --- {}".format(
        start_YEAR.strftime("%Y-%m-%d"), 
        end_YEAR.strftime("%Y-%m-%d")))

Used extraction dates: 2017-08-25 --- 2017-09-08


In [406]:
start_YEAR, end_YEAR = start_YEAR.strftime("%Y-%m-%d"), end_YEAR.strftime("%Y-%m-%d")

**Extract quotes during disaster's time interval**

In [362]:
df_disaster_start_end = df_time_interval(disaster_df_quotes, start_YEAR, end_YEAR,date_attr='date')

In [363]:
df_disaster_start_end.head(4)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2017-09-06-003914,advance this climate change agenda.,Rush Limbaugh,['Q319072'],2017-09-06 23:12:48,73,"[['Rush Limbaugh', '0.6842'], ['None', '0.2011...",['http://feeds.nydailynews.com/~r/nydnrss/spor...,E
1,2017-08-29-005991,"And actually, in the middle of the hurricane, ...",President Donald Trump,['Q22686'],2017-08-29 17:33:10,1,"[['President Donald Trump', '0.6877'], ['None'...",['http://southfloridagaynews.com/National/trum...,E
2,2017-09-01-010370,At the end of the day the congress will supply...,Bill Nelson,"['Q22639331', 'Q3272899', 'Q358437', 'Q4910326']",2017-09-01 12:46:19,1,"[['Bill Nelson', '0.7926'], ['None', '0.2074']]",['http://firstcoastnews.com/news/local/countie...,E
3,2017-08-29-016325,Catastrophic and epic flooding continued this ...,None,[],2017-08-29 07:32:24,2,"[['None', '0.8952'], ['Will Phillips', '0.1048']]",['http://www.arkansasonline.com/news/2017/aug/...,E


**Retrieve corresponding positive tag list and verify regex before filtering**

In [389]:
tags_pos_list = disaster_YEAR_pos.tags.values.tolist()

In [390]:
tags_pos_list

['(?=.*\\b([tT]exas|[lL]ouisiana|[cC]oasts?|U.?S.?A?|[bB]each(es)?)\\b)(?=.*\\b([rR]ain(ed|s|fall)?|[lL]andslides?)\\b)',
 '(?=.*\\b([tT]exas|[lL]ouisiana|[cC]oasts?|U.?S.?A?|[bB]each(es)?)\\b)(?=.*\\b([hH]urricanes?|[sS]torms?)\\b)',
 '(?=.*\\b([tT]exas|[lL]ouisiana|[cC]oasts?|U.?S.?A?)\\b)(?=.*\\b([fF]lood(waters?|s|ed|ing)?|[rR]ain(ed|s|fall)?)\\b)',
 '\\b(Harvey)\\b',
 '\\b([cC]ategory 4)\\b',
 '\\b([sS]an [jJ]os[ée] [iI]sland)\\b',
 '\\b(H.R. ?601)\\b',
 '\\b(NOAA)\\b',
 '\\b(Federal Emergency Management Agency|FEMA)\\b',
 '\\b(National Hurricane Center|NHC)\\b']

In [391]:
regex_pattern_pos = r'|'.join(tags_pos_list)

In [396]:
# Verify a regex by performing sample tests
sample_text = 'A boring sentence about experiments being performed Experimentally in an experimental setting.'
re.sub(r'\b([eE]xperiment(s|al|ally|)?)\b', 'FOUND PATTERN',sample_text)

'A boring sentence about FOUND PATTERN being performed FOUND PATTERN in an FOUND PATTERN setting.'

**Apply positive tags filtering (selecting quotes with desired patterns)**

In [367]:
filtered_for_pos = extract_quotes(df_disaster_start_end, regex_pattern_pos).sort_values(by='date')

/Users/gil/EPFL/4 - Fourth Year/Semester 1/ADA/Project/ada-2021-project-data-alchemy/disaster_extr_helpers.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  mask = df[field].str.contains(regex)


In [368]:
len(filtered_for_pos)

5023

In [369]:
filtered_for_pos.head(4)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
1506,2017-08-25-025209,facing first serious crises with Hurricane Har...,Matt Drudge,['Q1567226'],2017-08-25,5,"[['Matt Drudge', '0.5346'], ['President Donald...",['http://euronews.com/2017/08/25/hurricane-har...,E
30922,2017-08-25-055076,"If granted by the president, this request will...",None,[],2017-08-25,6,"[['None', '0.7802'], ['Greg Abbott', '0.2198']]",['http://keranews.org/post/harvey-makes-landfa...,E
10464,2017-08-25-104517,"Texas is about to get hit by a major hurricane,",Brock Long,['Q29964476'],2017-08-25,277,"[['Brock Long', '0.7948'], ['None', '0.1923'],...",['http://abcnews.go.com/US/hurricane-harvey-br...,E
9031,2017-08-25-077677,Keep on top of Hurricane Harvey. Don't make sa...,Chuck Grassley,['Q529294'],2017-08-25,105,"[['Chuck Grassley', '0.7189'], ['None', '0.200...",['https://www.vanguardngr.com/2017/08/disaster...,E


**For checking and verifying relevant and irrelevant quotes, then positive and negative tag lists can be adjusted accordingly**

In [372]:
# Filter based on quotation field
extract_quotes(filtered_for_pos, r'\b(some_pattern)\b', complement=False).quotation.values

array([], dtype=object)

In [374]:
# Filter based on urls field
extract_quotes(filtered_for_pos, r'some_pattern', field='urls', complement=False).quotation.values

array([], dtype=object)

**Apply negative tags filtering (removing quotes with undesired patterns from set of positive quotes)**

In [375]:
tag_neg_list = disaster_YEAR_neg.tags.values.tolist()

regex_pattern_neg = r'|'.join(tag_neg_list)

In [378]:
filtered_for_pos_then_neg = extract_quotes(filtered_for_pos, regex_pattern_neg, complement=True)

In [379]:
len(filtered_for_pos_then_neg)

5022

In [380]:
filtered_for_pos_then_neg.head(4)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
1506,2017-08-25-025209,facing first serious crises with Hurricane Har...,Matt Drudge,['Q1567226'],2017-08-25,5,"[['Matt Drudge', '0.5346'], ['President Donald...",['http://euronews.com/2017/08/25/hurricane-har...,E
30922,2017-08-25-055076,"If granted by the president, this request will...",None,[],2017-08-25,6,"[['None', '0.7802'], ['Greg Abbott', '0.2198']]",['http://keranews.org/post/harvey-makes-landfa...,E
10464,2017-08-25-104517,"Texas is about to get hit by a major hurricane,",Brock Long,['Q29964476'],2017-08-25,277,"[['Brock Long', '0.7948'], ['None', '0.1923'],...",['http://abcnews.go.com/US/hurricane-harvey-br...,E
9031,2017-08-25-077677,Keep on top of Hurricane Harvey. Don't make sa...,Chuck Grassley,['Q529294'],2017-08-25,105,"[['Chuck Grassley', '0.7189'], ['None', '0.200...",['https://www.vanguardngr.com/2017/08/disaster...,E


**In a few cases extra processing (based on URLs) was used**

In [381]:
apply_extra = False
apply_extra_to = filtered_for_pos_then_neg # or filtered_for_pos

In [382]:
if apply_extra:
    filtered_extra = storm_tags_2017_extra(apply_extra_to)
    print(len(filtered_extra))

**Double check parameters before writing to disk**

In [386]:
YEAR, disaster_type

(2017, 'storm')

In [387]:
output_df = filtered_for_pos_then_neg # Or filtered_extra or filtered_for_pos

In [388]:
output_df.head(4)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
1506,2017-08-25-025209,facing first serious crises with Hurricane Har...,Matt Drudge,['Q1567226'],2017-08-25,5,"[['Matt Drudge', '0.5346'], ['President Donald...",['http://euronews.com/2017/08/25/hurricane-har...,E
30922,2017-08-25-055076,"If granted by the president, this request will...",None,[],2017-08-25,6,"[['None', '0.7802'], ['Greg Abbott', '0.2198']]",['http://keranews.org/post/harvey-makes-landfa...,E
10464,2017-08-25-104517,"Texas is about to get hit by a major hurricane,",Brock Long,['Q29964476'],2017-08-25,277,"[['Brock Long', '0.7948'], ['None', '0.1923'],...",['http://abcnews.go.com/US/hurricane-harvey-br...,E
9031,2017-08-25-077677,Keep on top of Hurricane Harvey. Don't make sa...,Chuck Grassley,['Q529294'],2017-08-25,105,"[['Chuck Grassley', '0.7189'], ['None', '0.200...",['https://www.vanguardngr.com/2017/08/disaster...,E


In [308]:
write_df_to_disk(output_df, disaster_type, YEAR, additional_text='filtered')